In [2]:
import pandas as pd
import yfinance as yf
import requests
import io
import time
from scipy.stats import linregress

# Try to import yahooquery
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 1. HELPER FUNCTIONS (Credit Model)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score"""
    try:
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        elif 'Total Liabilities' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities'].iloc[0]
        else:
            return np.nan
        
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    try:
        limit = min(4, len(financials.columns))
        years = financials.columns[:limit]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        if len(margins) < 2: return "N/A"
        slope, _, _, _, _ = linregress(range(len(margins)), margins[::-1])
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"

def get_interest_coverage(financials):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        if 'Interest Expense' in financials.index: interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index: interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else: return 100.0
        
        interest = abs(interest)
        if interest == 0: return 100.0
        return round(ebit / interest, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        tax_prov = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        tax_rate = max(0.0, min(tax_prov / pre_tax, 0.30)) if pre_tax > 0 else 0.21
        nopat = ebit * (1 - tax_rate)

        equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        debt = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else 0
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested = equity + debt - cash
        if invested <= 0: return np.nan
        return round((nopat / invested) * 100, 2)
    except:
        return np.nan

# ==========================================
# 2. STEP 1: FETCH UNIVERSE (The Fix)
# ==========================================
def get_canadian_universe():
    print("--- STEP 1: Fetching Canadian Universe (TSX) ---")
    tickers = []
    
    # METHOD A: WIKIPEDIA
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        headers = {"User-Agent": "Mozilla/5.0"} 
        r = requests.get(url_ca, headers=headers)
        tables = pd.read_html(io.StringIO(r.text))
        
        df_ca = pd.DataFrame()
        
        # Smart Search: Look for ANY column that looks like a ticker symbol
        target_cols = ['Symbol', 'Ticker', 'Code']
        
        for t in tables:
            # Check if any of our target column names exist in this table
            found_col = next((col for col in target_cols if col in t.columns), None)
            
            if found_col:
                print(f"   [Debug] Found ticker table with column: '{found_col}'")
                df_ca = t
                # Standardize column name
                df_ca.rename(columns={found_col: 'Symbol'}, inplace=True)
                break
        
        if not df_ca.empty:
            ca_list = df_ca['Symbol'].astype(str).apply(lambda x: x.replace('.', '-') + ".TO").tolist()
            tickers.extend(ca_list)
            print(f"   -> Method A (Wiki) Success: Found {len(ca_list)} tickers.")
    except Exception as e:
        print(f"   -> Method A (Wiki) Failed: {e}")

    # METHOD B: EMERGENCY BACKUP LIST (If Wikipedia fails)
    if len(tickers) < 10:
        print("   -> WARNING: Wiki failed. Loading Emergency Backup List (Top 60 TSX Stocks).")
        # Hardcoded list of top TSX stocks to ensure script runs
        backup_list = [
            'RY.TO', 'TD.TO', 'SHOP.TO', 'ENB.TO', 'CNR.TO', 'CP.TO', 'BMO.TO', 'BNS.TO', 'TRI.TO', 'ATD.TO',
            'CSU.TO', 'CM.TO', 'TRP.TO', 'SU.TO', 'CNQ.TO', 'MFC.TO', 'IMO.TO', 'GIB-A.TO', 'POW.TO', 'QSR.TO',
            'BCE.TO', 'FTS.TO', 'WN.TO', 'DOL.TO', 'T.TO', 'L.TO', 'NA.TO', 'MG.TO', 'WCN.TO', 'CCO.TO',
            'AEM.TO', 'WPM.TO', 'FM.TO', 'OTEX.TO', 'RCI-B.TO', 'EMA.TO', 'SLF.TO', 'CVE.TO', 'MRU.TO', 'PPL.TO',
            'CTC-A.TO', 'SAP.TO', 'BIP-UN.TO', 'CAR-UN.TO', 'K.TO', 'TECK-B.TO', 'IVN.TO', 'H.TO', 'EFN.TO', 'GIL.TO'
        ]
        tickers.extend(backup_list)
        
    return list(set(tickers))

# ==========================================
# 3. STEP 2: CANADIAN FILTERS (Debug Mode)
# ==========================================
def filter_canada_universe(ticker_list):
    print(f"\n--- STEP 2: Applying Filters (Debug Mode) ---")
    
    # --- CRITERIA ---
    MIN_PRICE = 5.0
    MIN_CAP   = 100_000_000  # $100M 
    MIN_VOL   = 100_000      # 100k Volume
    
    valid_candidates = []
    chunk_size = 500 
    
    # Debug counters
    dropped_price = 0
    dropped_vol = 0
    dropped_cap = 0
    dropped_other = 0
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            # Fetch Price, Key Stats, and Financial Data modules
            data = yq.get_modules("summaryDetail defaultKeyStatistics price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): 
                    dropped_other += 1
                    continue
                
                # --- EXTRACT ---
                price_mod = data[symbol].get('price', {})
                summ_mod = data[symbol].get('summaryDetail', {})
                fin_mod = data[symbol].get('financialData', {})
                
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                
                # Optional extras
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                # --- DEBUG CHECK ---
                # Check why stocks fail (uncomment to see specific failures)
                # if symbol == 'RY.TO': print(f"\n   [DEBUG RY.TO]: Price {curr_price}, Vol {avg_vol}, Cap {mkt_cap}")

                if curr_price < MIN_PRICE:
                    dropped_price += 1
                    continue
                if mkt_cap < MIN_CAP:
                    dropped_cap += 1
                    continue
                if avg_vol < MIN_VOL:
                    dropped_vol += 1
                    continue
                
                # If solvency/profit filters enabled:
                if curr_ratio < 1.0 or op_margin < 0.001:
                    dropped_other += 1
                    continue

                # Add Survivor
                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Volume': avg_vol,
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin
                })
                        
        except Exception as e:
            print(f"Batch Error: {e}")
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete.")
    print(f"      Survivors: {len(df)}")
    print(f"      Dropped (Price < $5): {dropped_price}")
    print(f"      Dropped (Vol < 100k): {dropped_vol}")
    print(f"      Dropped (Cap < 100M): {dropped_cap}")
    print(f"      Dropped (Financial Health/Other): {dropped_other}")
    
    return df

# ==========================================
# 4. STEP 3: EXECUTION
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: 
        print("   CRITICAL: No candidates survived filters. Check your filter criteria.")
        return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model) on {len(candidates_df)} Stocks ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.1)
            
        try:
            stock = yf.Ticker(ticker)
            
            # 1. Option Check (Safety: skip if hard error)
            try:
                if not stock.options: continue 
            except:
                pass # Some valid stocks throw errors on options check, lenient here
            
            # 2. Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            if 'Financial' in info.get('sector', ''): continue
            
            # 3. Metrics
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Price': row['Price'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Region': 'Canada'
            }
            
            # 4. Buckets
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

# --- RUN IT ---
# 1. Get Universe
raw_tickers = get_canadian_universe()

# 2. Filter
filtered_df = filter_canada_universe(raw_tickers)

# 3. Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if fortress_df is not None and not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Canadian Picks) ---")
        display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    else:
        print("\nNo 'Fortress' stocks found (Criteria too strict?).")
        
    if moonshot_df is not None and not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS (Canadian Growth) ---")
        display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Canadian Universe (TSX) ---
   [Debug] Found ticker table with column: 'Ticker'
   -> Method A (Wiki) Success: Found 223 tickers.

--- STEP 2: Applying Filters (Debug Mode) ---
   Filtering batch 0 - 223...
   -> Filter complete.
      Survivors: 108
      Dropped (Price < $5): 11
      Dropped (Vol < 100k): 17
      Dropped (Cap < 100M): 0
      Dropped (Financial Health/Other): 87

--- STEP 3: Deep Analysis (Credit Model) on 108 Stocks ---
   Analyzing 105/108...
No 'Fortress' stocks found (Criteria too strict?).
